In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [3]:
#import the libraries
from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore')

In [4]:
#download the punkt package
nltk.download('punkt', quiet=True)

True

In [5]:
#get the article
article = Article('https://id.wikipedia.org/wiki/Kesehatan')
article.download()
article.parse()
article.nlp()
corpus = article.text

In [6]:
#print article
print(corpus)

Kesehatan adalah kondisi kesejahteraan fisik, mental, dan sosial yang lengkap dan bukan sekadar tidak adanya penyakit atau kelemahan.[1] Pemahaman tentang kesehatan telah bergeser seiring dengan waktu. Berkembangnya teknologi kesehatan berbasis digital telah memungkinkan setiap orang untuk mempelajari dan menilai diri mereka sendiri, dan berpartisipasi aktif dalam gerakan promosi kesehatan. Berbagai faktor sosial berpengaruh terhadap kondisi kesehatan, seperti perilaku individu, kondisi sosial, genetik dan biologi, perawatan kesehatan, dan lingkungan fisik.

Makna kesehatan telah berkembang seiring dengan waktu. Dalam perspektif model biomedis, definisi awal kesehatan difokuskan pada kemampuan tubuh untuk berfungsi. Kesehatan dipandang sebagai kondisi tubuh yang berfungsi normal yang dapat terganggu oleh penyakit dari waktu ke waktu.[2]

Pada tahun 1948, Organisasi Kesehatan Dunia (WHO) mendefinisikan kesehatan sebagai "kesejahteraan fisik, mental, dan sosial, dan bukan hanya tidak ada

In [7]:
#Tokenization
text = corpus
sentence_list = nltk.sent_tokenize(text)

In [8]:
#print sentence list
print(sentence_list)

['Kesehatan adalah kondisi kesejahteraan fisik, mental, dan sosial yang lengkap dan bukan sekadar tidak adanya penyakit atau kelemahan.', '[1] Pemahaman tentang kesehatan telah bergeser seiring dengan waktu.', 'Berkembangnya teknologi kesehatan berbasis digital telah memungkinkan setiap orang untuk mempelajari dan menilai diri mereka sendiri, dan berpartisipasi aktif dalam gerakan promosi kesehatan.', 'Berbagai faktor sosial berpengaruh terhadap kondisi kesehatan, seperti perilaku individu, kondisi sosial, genetik dan biologi, perawatan kesehatan, dan lingkungan fisik.', 'Makna kesehatan telah berkembang seiring dengan waktu.', 'Dalam perspektif model biomedis, definisi awal kesehatan difokuskan pada kemampuan tubuh untuk berfungsi.', 'Kesehatan dipandang sebagai kondisi tubuh yang berfungsi normal yang dapat terganggu oleh penyakit dari waktu ke waktu.', '[2]\n\nPada tahun 1948, Organisasi Kesehatan Dunia (WHO) mendefinisikan kesehatan sebagai "kesejahteraan fisik, mental, dan sosial,

In [9]:
class tokenize():
    def WordTokenize(self, sentence, stopword=None, removepunct=False, splitby='space'):
        #isi param = sentence:kalimat
        #stopword:listkatatidakpenting
        #removepunct:tandabaca'
        #splitby:pisahberdasarkan
        if splitby.strip().lower()=='space':
            words = re.split(r'\s',sentence)
        elif splitby.strip().lower()=='word':
            words = re.split('(\w+)?', sentence)
        else:
            raise NotImplementedError
    
        #jika remove punct true maka tanda baca akan dihapus
        if removepunct:
            table = string.maketrans("","")
            words = [z.translate(table,string.punctuation).strip() for z in words]
    
        #hapus seluruh empty char pada list
        words = [x.strip().lower() for x in words if x.strip()]
    
        return words

    def CharTokenize(self,word):
        # untuk kata
        return list(word)

In [10]:
tkKlmt = tokenize()
klmt = tkKlmt.WordTokenize("dia pergi ke !! pasar memang. :)")
klmt2 = tkKlmt.WordTokenize("mereka pergi2nya, ke kota lagi ntar")
kata = tkKlmt.CharTokenize("aku")
print(klmt)
print(klmt2)
print(kata)

['dia', 'pergi', 'ke', '!!', 'pasar', 'memang.', ':)']
['mereka', 'pergi2nya,', 'ke', 'kota', 'lagi', 'ntar']
['a', 'k', 'u']


In [11]:
#fungsi untuk memberikan respon terhadap user
def greeting_response(text):
    text = text.lower()
    
    bot_greetings = ['hi', 'hey', 'hello', 'halo']
    user_greetings = ['hi', 'hey', 'hello', 'halo']
    
    
    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)

In [12]:
def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))
    
    x = list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    
    return list_index

In [13]:
#membuat response dari bot
def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response = ''
    cm = CountVectorizer().fit_transform(sentence_list)
    similarity_scores = cosine_similarity(cm[-1], cm)
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    response_flag = 0
    
    j = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0:
            bot_response = bot_response+' '+sentence_list[index[i]]
            response_flag = 1
            j = j+1
        if j > 2:
            break
            
    if response_flag == 0:
        bot_response = bot_response+' '+"I apologize, I don't understant."
        
    sentence_list.remove(user_input)
    
    return bot_response

In [ ]:
#memulai percapakapan dengan bot
print('Doc Bot: I am Doctor Bot or Doc Bot for short. I will answer your queries about Chronic Kidney Disease. If you want to exit, type bye.')

exit_list = ['exit', 'see you later', 'bye', 'quit', 'break']

while(True):
    user_input = input()
    if user_input.lower() in exit_list:
        print('Doc Bot: Chat with you later !')
        break
    else:
        if greeting_response(user_input) != None:
            print('Doc Bot: '+greeting_response(user_input))
        else:
            print('Doc Bot: '+bot_response(user_input))

Doc Bot: I am Doctor Bot or Doc Bot for short. I will answer your queries about Chronic Kidney Disease. If you want to exit, type bye.
hi
Doc Bot: hello
kesehatan adalah
Doc Bot:  Seri Laporan Kesehatan Dunia yang diterbitkan oleh WHO berfokus pada masalah kesehatan global, termasuk akses untuk mendapatkan pelayanan kesehatan dan upaya meningkatkan kesehatan masyarakat, terutama di negara-negara berkembang. [6]

Gerakan promosi kesehatan memungkinkan kesehatan untuk diajarkan, dipelajari, dan diperkuat. [20]

Kesehatan global [ sunting | sunting sumber ]

Kesehatan global merupakan penelitian dan tindakan kolaboratif lintas negara untuk mempromosikan kesehatan untuk semua.
